In [ ]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import optuna 
import torch
import torch.nn as nn
from torch.utils.data import DataLoader 
from sklearn.model_selection import train_test_split
from DataEncoder import encode_pad_event, encode_pad_sequence, scale_time_differences, encode_label_event, node_time_list, event_transition_edge, scale_time_differences_fast_fixed, length_stratified_split
from GATConvStatusEmb import prepare_data_core_2edges, prepare_data_y, CustomDataset, EarlyStopping, train, evaluate, custom_collate_fn, DualGAT2EdgesModel
from GATConvStatusEmb import predict, top_k_accuracy, predict_per_sequence, average_bleu_score, compute_dls_and_exact_match, sequence_level_top_k_accuracy, analyze_sequence_errors, predict_per_sequence_with_probs, sequence_level_top_k_analysis, show_error_sequences
import os 
import shutil

In [ ]:
#event = pd.read_csv("../output/BPI12.csv")
event = pd.read_csv("../output/BPI12w.csv")
#event = pd.read_csv("../output/BPI13i.csv")
#event = pd.read_csv("../output/BPI13c.csv")
#event = pd.read_csv("../output/helpdesk.csv")

In [ ]:
# check the size of sequence
shortest_sequence = event.groupby('sequence').size().min()
longest_sequence = event.groupby('sequence').size().max()
print('shortest_sequence:', shortest_sequence)
print('longest_sequence:', longest_sequence)
event = event[event.groupby('sequence')['sequence'].transform('size') >= 2].reset_index(drop=True)

In [ ]:
#core_event = 'event_label'
core_event = 'event' #substatus
case_index = 'sequence'

In [ ]:
core_encode, y_encode, core_size, output_size, le_event = encode_label_event(event, core_event, case_index)

In [ ]:
#event['ec1'] = event['ec1'].astype(str) 
cat_col_event = ['ec1']
num_col_event = []
#Bpi13
#cat_col_event = ['ec1', 'ec2', 'ec4', 'ec5']
#num_col_seq = []
#helpdesk
#cat_col_event = ['ec1']
#num_col_event = []
event_encode = encode_pad_event(event, cat_col_event, num_col_event, case_index, cat_mask = True, num_mask = True, eos = False)

In [ ]:
sequence = event[['sequence','sn1']].groupby('sequence').first()
#BPI13
#sequence = event[['sequence','sc3', 'sc1', 'sc2']].groupby('sequence').first()
#sequence = event[['sequence','sc1']].groupby('sequence').first()
sequence = sequence.reset_index()
#sequence = event.groupby('sequence').apply(lambda x: x.iloc[prefix_size - 1:]).reset_index(drop=True)

In [ ]:
cat_col_seq = []
num_col_seq = ['sn1']
#BPI13
#cat_col_seq = ['sc1','sc2','sc3']
#num_col_seq = []
#helpdesk
#cat_col_seq = ['sc1']
#num_col_seq = []
sequence_encode = encode_pad_sequence(sequence, cat_col_seq, num_col_seq)

In [ ]:
status = 'event'
event_trans_edge, le_edge, trans_size = event_transition_edge(event, sequence, status, case_index)

In [ ]:
start_time_col = 'time'
#start_time_col = 'Start Timestamp'
scaled_time_diffs = scale_time_differences_fast_fixed(event, sequence, start_time_col, case_index)

In [ ]:
max_num_events = event_encode.shape[1]
# Expand sequence features to match the shape of event features
sequence_features_expanded = np.expand_dims(sequence_encode, axis=1)
sequence_features_expanded = np.repeat(sequence_features_expanded, max_num_events, axis=1)

# Combine event and sequence features
combined_features = np.concatenate((event_encode, sequence_features_expanded), axis=2)

In [ ]:
num_sequences = event_encode.shape[0]
num_event_features = combined_features.shape[2]
num_embedding_features = core_size

In [ ]:
event_feature_list = prepare_data_core_2edges(combined_features, core_encode, scaled_time_diffs, event_trans_edge)
y_list = prepare_data_y(combined_features, y_encode)

In [ ]:
# Use stratified sampling over sequence length to preserve distributional characteristics
train_indices, test_indices = length_stratified_split(event_feature_list, test_size=0.2, n_bins=10)

# Split the data
train_event_features = [event_feature_list[i] for i in train_indices]
test_event_features = [event_feature_list[i] for i in test_indices]
train_y = [y_list[i] for i in train_indices]
test_y = [y_list[i] for i in test_indices]

# Print statistics
train_lengths = [event_feature_list[i].x.shape[0] for i in train_indices]
test_lengths = [event_feature_list[i].x.shape[0] for i in test_indices]

print(f"Train set: {len(train_indices)} samples")
print(f"Train length range: {min(train_lengths)} - {max(train_lengths)}")
print(f"Train length mean: {np.mean(train_lengths):.2f}")

print(f"\nTest set: {len(test_indices)} samples") 
print(f"Test length range: {min(test_lengths)} - {max(test_lengths)}")
print(f"Test length mean: {np.mean(test_lengths):.2f}")

In [ ]:
train_dataset = CustomDataset(train_event_features, train_y)
test_dataset = CustomDataset(test_event_features, test_y)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# parameters
embedding_dims = 64
gat_hidden_dim_event = 32
gat_hidden_dim_embed = 128
gat_hidden_dim_concat = 256
output_dim = output_size  # vocab size
num_heads = 4
num_edge_types = trans_size
edge_type_dim = 32

model = DualGAT2EdgesModel(
    num_event_features=num_event_features,
    num_embedding_features=num_embedding_features,
    embedding_dims=embedding_dims,
    gat_hidden_dim_event=gat_hidden_dim_event,
    gat_hidden_dim_embed=gat_hidden_dim_embed,
    gat_hidden_dim_concat=gat_hidden_dim_concat,
    output_dim=output_dim,
    num_heads=num_heads,
    num_edge_types=num_edge_types,
    edge_type_dim=edge_type_dim
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=-1)  # ignore padding

In [ ]:
batch_size = 16  
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)

In [ ]:
early_stopping = EarlyStopping(patience=3, delta=0.0)
num_epochs = 10

# Filepath to save model
model_path = "../output/models/BPI12w_transedge_es.pt"

config = {
    'num_event_features': num_event_features,
    'num_embedding_features': num_embedding_features,
    'embedding_dims': embedding_dims,
    'gat_hidden_dim_event': gat_hidden_dim_event,
    'gat_hidden_dim_embed': gat_hidden_dim_embed,
    'gat_hidden_dim_concat': gat_hidden_dim_concat,
    'output_dim': output_dim,
    'num_heads': num_heads,
    'num_edge_types': num_edge_types,
    'edge_type_dim': edge_type_dim
}

for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | Test Loss: {test_loss:.4f} Acc: {test_acc:.4f}")
    
    if early_stopping(test_loss):
        print("Early stopping triggered.")
        break

    if early_stopping.best_loss_updated:
        print(f"New best model at epoch {epoch+1}, saving to {model_path}")
        best_model_saved = True
         # Save state dict and config
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'train_acc': train_acc,
            'test_loss': test_loss,
            'test_acc': test_acc,
            'config': config
        }, model_path)
        
if not early_stopping.early_stop and not best_model_saved:
        print("Training completed without early stopping. Saving final model.")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'train_acc': train_acc,
            'test_loss': test_loss,
            'test_acc': test_acc,
            'config': config
        }, model_path)

In [ ]:
preds, labels, outs = predict(model, test_loader, device)

# Check some predictions
for i in range(10):
    print(f"Predicted: {preds[i].item()} | Actual: {labels[i].item()}")

In [ ]:
top1 = top_k_accuracy(outs, labels, k=1)
top3 = top_k_accuracy(outs, labels, k=3)
top5 = top_k_accuracy(outs, labels, k=5)

print(f"Top-1 Acc: {top1:.4f} | Top-3 Acc: {top3:.4f} | Top-5 Acc: {top5:.4f}")

In [ ]:
preds_seq, labels_seq = predict_per_sequence(model, test_loader, device)

In [ ]:
bleu = average_bleu_score(preds_seq, labels_seq)
print(f"Average BLEU Score: {bleu:.4f}")

In [ ]:
avg_dls, exact_acc = compute_dls_and_exact_match(preds_seq, labels_seq)
print(f"Damerau-Levenshtein Similarity (avg): {avg_dls:.4f}")
print(f"Exact Match Accuracy: {exact_acc:.4f}")

In [ ]:
# Usage
seq_top3_acc = sequence_level_top_k_accuracy(model, test_loader, device, k=3)
print(f"Sequence-Level Top-3 Accuracy: {seq_top3_acc:.4f}")
seq_top5_acc = sequence_level_top_k_accuracy(model, test_loader, device, k=5)
print(f"Sequence-Level Top-5 Accuracy: {seq_top5_acc:.4f}")

In [ ]:
# Usage example
pos_errors, error_types, seq_stats = analyze_sequence_errors(model, test_loader, device, k=3)

In [ ]:
# First get predictions with probabilities
all_preds, all_labels, all_topk = predict_per_sequence_with_probs(model, test_loader, device, k=3)

# Then analyze sequence-level top-k accuracy
topk_acc, error_stats = sequence_level_top_k_analysis(all_topk, all_labels)

print(f"Sequence-Level Top-3 Accuracy: {topk_acc:.4f}")
print("\nError Analysis:")
print(f"- Most error-prone positions: {error_stats['position_errors']}")
print(f"- Top prediction mistakes: {error_stats['top_errors']}")

In [ ]:
show_error_sequences(all_topk, all_labels, num=3)

In [ ]:
from sklearn.metrics import classification_report
unique_true = np.unique(labels.cpu().numpy())
unique_preds = np.unique(preds.cpu().numpy())

print("Classes in true labels:", unique_true)
print("Classes in predictions:", unique_preds)
print("Missing classes:", set(unique_true) - set(unique_preds))

actual_classes = np.union1d(unique_true, unique_preds)
report = classification_report(
    labels.cpu().numpy(),
    preds.cpu().numpy(),
    target_names=[f"Class_{i}" for i in actual_classes] ,
    digits=4
)
print(report)

In [ ]:
# Load model state
checkpoint = torch.load(model_path, map_location=device)
config = checkpoint['config']

# Rebuild the model using the same config
model = DualGAT2EdgesModel(
    num_event_features=config['num_event_features'],
    num_embedding_features=config['num_embedding_features'],
    embedding_dims=config['embedding_dims'],
    gat_hidden_dim_event=config['gat_hidden_dim_event'],
    gat_hidden_dim_embed=config['gat_hidden_dim_embed'],
    gat_hidden_dim_concat=config['gat_hidden_dim_concat'],
    output_dim=config['output_dim'],
    num_heads=config['num_heads'],
    num_edge_types=config['num_edge_types'],
    edge_type_dim=config['edge_type_dim']
)

model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)

# Rebuild optimizer (if needed)
optimizer = torch.optim.Adam(model.parameters())
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Recover attention and event IDs
start_epoch = checkpoint['epoch'] + 1 

for epoch in range(start_epoch, 10):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | Test Loss: {test_loss:.4f} Acc: {test_acc:.4f}")
    
    if early_stopping(test_loss):
        print("Early stopping triggered.")
        break

    if early_stopping.best_loss_updated:
        print(f"New best model at epoch {epoch+1}, saving to {model_path}")
        best_model_saved = True
         # Save state dict and config
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'train_acc': train_acc,
            'test_loss': test_loss,
            'test_acc': test_acc,
            'config': config
        }, model_path)
        
if not early_stopping.early_stop and not best_model_saved:
        print("Training completed without early stopping. Saving final model.")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'train_acc': train_acc,
            'test_loss': test_loss,
            'test_acc': test_acc,
            'config': config
        }, model_path)